In [27]:
import torch 
import torch.nn as nn
import wandb

In [28]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="DL With PyTorch",   
)

idx,▁▁▁
idx,0
model,"Conv2d(3, 10, kernel..."
msg,None found for Gradi...
param,output


In [29]:
class myNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Conv2d(3,10,2, stride = 2)
    self.relu = nn.ReLU()
    self.flatten = lambda x: x.view(-1)
    self.fc1 = nn.Linear(160,5)
   
  
  def forward(self, x):
    x = self.relu(self.conv(x))
    return self.fc1(self.flatten(x))

In [30]:

net = myNet()
wandb.log({"model":net})

def hook_fn(model, inp, out):
    wandb.log({"model":model})
    for idx, grad in enumerate(inp):
        try:
          wandb.log({"param":"input", "idx":idx, "grad_shape":grad.shape, "grad":grad})
        except AttributeError: 
          wandb.log ({"msg":"None found for Gradient"})

    for idx, grad in enumerate(out):  
        try:
          wandb.log({"param":"output", "idx":idx, "grad_shape":grad.shape, "grad":grad})
        except AttributeError: 
          wandb.log ({"msg":"None found for Gradient"})



In [31]:
net.conv.register_full_backward_hook(hook_fn)
net.fc1.register_full_backward_hook(hook_fn)
inp = torch.randn(1,3,8,8)
out = net(inp)

(1 - out.mean()).backward()

In [32]:
wandb.save()

True